In [ ]:
# Install and Import all required libraries

%pip install openai pandas cassandra-driver cassio langchain PyMuPDF
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
from getpass import getpass
import os

In [ ]:
# Collect all information required to connect to your Astra Database and OpenAI API. Also specify the model_id for generating the embeddings.

ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")
ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API Key: ")

In [ ]:

# Connect to your Astra Database

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH
    },
    auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN),
)

session = cluster.connect()

In [ ]:
# Read the PDF file and convert it into loadable format
from langchain.document_loaders import PyMuPDFLoader

pdffilepath = input("Please provide the full path to your PDF file: ")
loader = PyMuPDFLoader(pdffilepath)
documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import Cassandra

table_name = "pdftexttable"
pdftextsearchdb = Cassandra.from_documents(
    documents=texts,
    embedding=embeddings,
    table_name=table_name,
    keyspace=ASTRA_DB_KEYSPACE,
    session=session,
)

In [ ]:
query = input("Please enter your search query: ")

docs = pdftextsearchdb.similarity_search(query, k=3)

print(docs[0].page_content)
print(docs[1].page_content)
print(docs[2].page_content)